---
title: "motivation"
execute:
  freeze: auto  # re-render only when source changes
---

See below a graph of tree transpiration as a function of time. The dots represent actual measurements from several transpiration chambers placed on individual tree branches. It might be hard to find a pattern amidst all these scattered points, so we can ask "what is the average behavior of these points across time?".

When we learned resampling, we did exactly that, and we saw examples of how to take daily or monthly temperature averages. Downsampling, however, decreases the temporal frequency of our data (e.g. from daily to monthly). How can we take averages of our data without decreasing its frequency?

## moving window

One common solution is to analyze our data in small windows, and slide this window from left to right in order to get tbe aggregate behavior of our data in a small time interval. For daily data, if we have have a 1-month long window sliding from January to December in 1-month steps, then we exactly resampled our data into monthly bins, and also decreased its frequency from 1 day to 1 month. If the window slides by only 1 day, however, then we get a (slightly smaller) array also with a 1-day frequency. This is the central idea of a moving window.

## example

Temperature in jerusalem 2019

## we don't need to take averages

There are many ways of aggregating your data inside a window, and we will see some of them in the following sections.